# Imports

In [1]:
import numpy as np
import pandas as pd
import finananal as finan
import tensorflow as tf

import datetime as dt
import matplotlib.dates as mdates

from datetime import datetime
from sklearn import svm

import json
import pickle

# Preparing the notebook

In [2]:
pd.options.display.max_columns = None

# Loading the data for the techincal analisis

In [3]:
sp500 = pd.read_csv('Future/^GSPC_future_real.csv', parse_dates=True, index_col=0)
sp500 = sp500.asfreq(freq='B', method = 'pad')
sp500 = sp500.drop('Adj Close', axis =1)
sp500.head()

,Open,High,Low,Close,Volume
Date,,,,,
2019-11-20,3114.659912,3118.969971,3091.409912,3108.459961,4034890000
2019-11-21,3108.489990,3110.110107,3094.550049,3103.540039,3720560000
2019-11-22,3111.409912,3112.870117,3099.260010,3110.290039,3226780000
2019-11-25,3117.439941,3133.830078,3117.439941,3133.639893,3511530000
2019-11-26,3134.850098,3142.689941,3131.000000,3140.520020,4595590000


In [4]:
sp500['Close Change'] = sp500.diff()['Close']

In [5]:
sp500

,Open,High,Low,Close,Volume,Close Change
Date,,,,,,
2019-11-20,3114.659912,3118.969971,3091.409912,3108.459961,4034890000,NaN
2019-11-21,3108.489990,3110.110107,3094.550049,3103.540039,3720560000,-4.919922
2019-11-22,3111.409912,3112.870117,3099.260010,3110.290039,3226780000,6.750000
2019-11-25,3117.439941,3133.830078,3117.439941,3133.639893,3511530000,23.349854
2019-11-26,3134.850098,3142.689941,3131.000000,3140.520020,4595590000,6.880127
...,...,...,...,...,...,...
2020-04-14,2805.100098,2851.850098,2805.100098,2846.060059,5567400000,84.430176
2020-04-15,2795.639893,2801.879883,2761.540039,2783.360107,5203390000,-62.699952
2020-04-16,2799.340088,2806.510010,2764.320068,2799.550049,5179990000,16.189942


# Loading fundamental analisis data

In [6]:
sp500_fun_an = pd.read_csv('Future/SP500_FUN_AN', parse_dates=True, index_col=0)
sp500_fun_an

,GDPC1,CP,UMCSENT,WRMFSL,EUANDH,PANDI,ANFCI,INDPRO,GOLD,DCOILWTICO,Bullish,Neutral,Bearish,S&P 500 Weekly Close
DATE,,,,,,,,,,,,,,
1989-01-01,9101.508,275.100,97.9,238.3,0.20,0.23,0.32,64.0153,410.25,NaN,0.240000,0.540000,0.220000,277.72
1989-01-02,9101.508,275.100,97.9,239.8,0.20,0.23,0.32,64.0153,410.25,NaN,0.240000,0.540000,0.220000,277.72
1989-01-03,9101.508,275.100,97.9,239.8,0.20,0.23,0.32,64.0153,411.60,NaN,0.240000,0.540000,0.220000,277.72
1989-01-04,9101.508,275.100,97.9,239.8,0.20,0.23,0.32,64.0153,409.35,NaN,0.240000,0.540000,0.220000,277.72
1989-01-05,9101.508,275.100,97.9,239.8,0.20,0.23,0.32,64.0153,407.25,NaN,0.240000,0.540000,0.220000,277.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-08,19221.970,1903.753,101.0,1125.5,-1.23,-2.72,0.40,103.6635,1562.80,24.97,0.342391,0.160326,0.497283,2470.50
2020-04-09,19221.970,1903.753,101.0,1125.5,-1.23,-2.72,0.40,103.6635,1562.80,22.90,0.365994,0.187320,0.446686,2749.98
2020-04-10,19221.970,1903.753,101.0,1125.5,-1.23,-2.72,0.26,103.6635,1562.80,22.90,0.365994,0.187320,0.446686,2749.98


# Defining the period of time

In [7]:
start = dt.datetime(2020, 1, 1)
end = dt.datetime(2020, 4, 20)
start_plot = dt.datetime(2019, 6, 1)
end_plot = dt.datetime(2020, 1, 9)

# Generation of the data

In [8]:
all_features, all_y = finan.generator(sp500, sp500_fun_an, start = start, end = end, days_an = 45)

/home/marcelo/anaconda3/envs/jupyter/lib/python3.8/site-packages/ta/trend.py:567: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i]/self._trs[i])
/home/marcelo/anaconda3/envs/jupyter/lib/python3.8/site-packages/ta/trend.py:571: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i]/self._trs[i])


In [9]:
all_features

{2:                   Close      Volume  Close Change        ADX    ADX neg  \
 Date                                                                      
 2020-01-01  3230.780029  2893810000      0.000000  57.500182  17.382678   
 2020-01-02  3257.850098  3458250000     27.070069  67.784340   7.214134   
 2020-01-03  3234.850098  3461290000    -23.000000  35.955847  25.260073   
 2020-01-06  3246.280029  3674070000     11.429931  36.829283  24.556727   
 2020-01-07  3237.179932  3420380000     -9.100097  37.266001  16.943507   
 ...                 ...         ...           ...        ...        ...   
 2020-04-14  2846.060059  5567400000     84.430176  59.442458  12.950656   
 2020-04-15  2783.360107  5203390000    -62.699952  36.660770  32.719925   
 2020-04-16  2799.550049  5179990000     16.189942  20.202973  21.647697   
 2020-04-17  2874.560059  5792140000     75.010010  46.432405   9.502850   
 2020-04-20  2823.159912  2935031893    -51.400147  44.464119  13.743130   
 
       

In [10]:
all_y

Date
2020-01-01     0.000000
2020-01-02    27.070069
2020-01-03   -23.000000
2020-01-06    11.429931
2020-01-07    -9.100097
                ...    
2020-04-14    84.430176
2020-04-15   -62.699952
2020-04-16    16.189942
2020-04-17    75.010010
2020-04-20   -51.400147
Freq: B, Name: Close Change, Length: 79, dtype: float64

# Preparation of the data

In [11]:
frames = finan.multiplexor(all_features)

In [12]:
frames[8]

,Close,Volume,Close Change,ADX,ADX neg,ADX pos,EMA,DPO,Williams %R,ROC,KAMA,MFI,RSI,TSI,Stock Osc,Stock Osc sig,ADI,Chainkin MF,EoM,EoM SMA,Bol Band mavg,Bol Band high,Bol Band low,ATR,Donchian hi-band,Donchian hi-ind,Donchian lo-band,Donchian lo-ind,GDPC1,CP,UMCSENT,WRMFSL,EUANDH,PANDI,ANFCI,INDPRO,GOLD,DCOILWTICO,Bullish,Neutral,Bearish
0,3288.129883,3.456380e+09,22.779785,86.425887,1.026895,30.294302,3280.605813,-2.040040,-0.0,0.410112,3261.236550,51.915001,84.552021,33.751217,100.0,72.675773,3.282041e+10,0.231985,3.622009,2.603104,3276.739991,3299.519775,3253.960206,23.034689,3288.129883,1.0,3265.350098,0.0,19221.97,1903.753,99.8,984.8,-0.04,-0.3,-0.63,109.083,1549.9,58.17,0.330688,0.37037,0.298942
1,3288.129883,3.456380e+09,22.779785,62.494831,3.702656,31.578336,3275.556686,-1.360026,-0.0,1.078367,3260.780179,68.872623,79.651704,33.751217,100.0,86.625901,3.282041e+10,0.450908,3.622009,3.672751,3276.059977,3294.758776,3257.361178,23.459978,3288.129883,1.0,3265.350098,0.0,19221.97,1903.753,99.8,984.8,-0.04,-0.3,-0.63,109.083,1549.9,58.17,0.330688,0.37037,0.298942
2,3288.129883,3.456380e+09,22.779785,48.404613,5.941187,31.505125,3271.441748,-17.257446,-0.0,1.573899,3257.313070,77.087882,76.714680,33.751217,100.0,87.888922,3.282041e+10,0.351903,3.622009,5.450720,3270.307495,3295.984847,3244.630143,23.457805,3288.129883,1.0,3253.050049,0.0,19221.97,1903.753,99.8,984.8,-0.04,-0.3,-0.63,109.083,1549.9,58.17,0.330688,0.37037,0.298942
3,3288.129883,3.456380e+09,22.779785,41.741411,7.524371,31.452029,3267.964384,-10.631934,-0.0,1.289163,3248.975402,81.545666,74.953901,33.751217,100.0,90.915869,3.282041e+10,0.236116,3.622009,4.939257,3263.681983,3298.750770,3228.613196,23.196292,3288.129883,1.0,3237.179932,0.0,19221.97,1903.753,99.8,984.8,-0.04,-0.3,-0.63,109.083,1549.9,58.17,0.330688,0.37037,0.298942
4,3288.129883,3.456380e+09,22.779785,40.588365,8.654285,31.569573,3264.925858,-23.601725,-0.0,1.647056,3250.418658,84.733915,73.888192,33.751217,100.0,90.915869,3.282041e+10,0.362933,3.622009,3.604073,3260.781657,3295.322757,3226.240557,22.850739,3288.129883,1.0,3237.179932,0.0,19221.97,1903.753,99.8,984.8,-0.04,-0.3,-0.63,109.083,1549.9,58.17,0.330688,0.37037,0.298942
5,3288.129883,3.456380e+09,22.779785,40.372679,9.497129,31.811733,3262.175933,-19.897217,-0.0,0.929441,3245.418943,72.877812,73.219046,33.751217,100.0,90.935640,3.282041e+10,0.318991,3.622009,1.851992,3257.077149,3293.846806,3220.307491,22.505879,3288.129883,1.0,3234.850098,0.0,19221.97,1903.753,99.8,984.8,-0.04,-0.3,-0.63,109.083,1549.9,58.17,0.330688,0.37037,0.298942
6,3288.129883,3.456380e+09,22.779785,40.011056,10.155933,32.117165,3259.607844,-10.893738,-0.0,1.775109,3247.003696,76.218888,72.775250,33.751217,100.0,91.252043,3.282041e+10,0.399812,3.622009,3.660330,3257.173767,3291.572432,3222.775103,22.190861,3288.129883,1.0,3234.850098,0.0,19221.97,1903.753,99.8,984.8,-0.04,-0.3,-0.63,109.083,1549.9,58.17,0.330688,0.37037,0.298942
7,3288.129883,3.456380e+09,22.779785,39.296582,10.733056,32.426975,3257.150789,-7.961101,-0.0,1.775109,3245.304766,76.218888,72.461780,33.751217,100.0,91.252043,3.282041e+10,0.447023,3.622009,3.253627,3254.241130,3290.669217,3217.813043,21.911651,3288.129883,1.0,3230.780029,0.0,19221.97,1903.753,99.8,984.8,-0.04,-0.3,-0.63,109.083,1549.9,58.17,0.330688,0.37037,0.298942
8,3288.129883,3.456380e+09,22.779785,37.627147,11.323886,32.691912,3254.760843,-17.044922,-0.0,2.074940,3245.651568,69.147432,72.226200,33.751217,100.0,91.252043,3.282041e+10,0.486158,3.622009,2.460819,3251.895020,3289.210664,3214.579375,21.674183,3288.129883,1.0,3230.780029,0.0,19221.97,1903.753,99.8,984.8,-0.04,-0.3,-0.63,109.083,1549.9,58.17,0.330688,0.37037,0.298942
9,3288.129883,3.456380e+09,22.779785,36.410243,11.877373,32.969038,3252.412713,-14.262651,-0.0,1.484863,3243.923890,63.053251,72.039278,33.751217,100.0,91.252043,3.282041e+10,0.396318,3.622009,1.325810,3249.112749,3288.805463,3209.420034,21.483634,3288.129883,1.0,3221.290039,0.0,19221.97,1903.753,99.8,984.8,-0

In [13]:
# f = open("multiplexed/X_2004_1closech.pkl","wb")
# pickle.dump(frames,f)
# f.close()

In [14]:
# f = open("multiplexed/y_2004_1closech.pkl","wb")
# pickle.dump(all_y,f)
# f.close()

In [15]:
sp500_past = pd.read_pickle(r'multiplexed/X_1604_1closech.pkl')

In [16]:
y_past = pd.read_pickle(r'multiplexed/y_1604_1closech.pkl')

In [ ]:
def desnorm(df_norm, df):
    df_desnorm = []
    for i in range(len(df_norm)):
        vect_desnorm = df
        df_desnorm.append(df_norm * (vect_desnorm.max()-vect_desnorm.min()) + vect_desnorm.min())
    return df_desnorm